In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
file_dir = '/kaggle/input/learn-ai-bbc/'
train_file = file_dir + 'BBC News Train.csv'
test_file = file_dir + 'BBC News Test.csv'
submit_file = file_dir + 'BBC News Sample Solution.csv'


In [ ]:
data_train = pd.read_csv(train_file)

In [ ]:
data_train.head()

In [ ]:
data_train.Category.value_counts()

In [ ]:
#max length of Text present in data
srt =[]
for s in data_train['Text']:
    srt.append(len(s.split(' ')))
srt.sort()
srt[-1]

****Split data in train and test********

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
X = data_train['Text'].values
# X = data_train.iloc[0:2].values
y = data_train['Category'].values

y_t = np.array(y)
# integer encode
le = LabelEncoder()
integer_encoded = le.fit_transform(y_t)
# binary encode
ohe = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_t1 = ohe.fit_transform(integer_encoded)
print(y_t1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_t1, test_size = 0.20, random_state = 1000)

In [ ]:
print(X_train.size)
print(X_test.size)
print(len(data_train))

**Tokenize Data**

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(X_train)

X_train1 = tokenizer.texts_to_sequences(X_train)
X_test1 = tokenizer.texts_to_sequences(X_test)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [ ]:
print(X_train[1:6])

In [ ]:
print(X_train1[0])

**PADS Sequence**

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# have to check the value of this hyperparameter - calculated above 3519
maxlen = 3100

In [ ]:
X_train1 = pad_sequences(X_train1, padding= 'post', maxlen = maxlen)
X_test1 = pad_sequences(X_test1, padding= 'post', maxlen = maxlen)

In [ ]:
print(X_train1[0])

**Model Creation**

In [ ]:
import tensorflow as tf
m = tf.keras.models
layers = tf.keras.layers

In [ ]:
model = m.Sequential()

In [ ]:
# vocab size is 22811
# input_length is size of review text after tokenization and pad sequance
embedding_dim = 75


model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim,
                           input_length=maxlen))

#model.add(layers.Flatten())
model.add(layers.GlobalMaxPool1D())
#model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(1024, activation='relu'))
#model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))


model.summary()

In [ ]:
adam = tf.keras.optimizers.Adam(lr=0.0005)
model.compile(optimizer= adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train1, y_train,
                    epochs=18,verbose=True,
                    validation_data=(X_test1, y_test),
                    batch_size=10)

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    
    vocab_size = len(word_index) + 1 
    # Adding again 1 because of reserved 0 index
    
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as file:
        
        for line in file:
            
            # Fetching word and weights of that word from pre-trained file
            word, *vector = line.split()
            
            # putting word only if it exist in text for which model is created
            if word in word_index:
                # Find token number of each word from Toekenzier
                idx = word_index[word]
                #print("{} {} ".format(word,idx))
                # Create matrix with toekn and its vector from pre-trained weights
                embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


In [ ]:
embedding_matrix = create_embedding_matrix(filePath,
                                          tokenizer.word_index,
                                          embedding_dim)


model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim,
                           input_length=maxlen))